In [1]:
from datetime import timedelta, datetime
import config
import uuid
from pandasql import sqldf
from sqlalchemy.dialects.postgresql import insert
import pandas as pd
import numpy as np

In [2]:
SELECT_TYPE10_QRY = """SELECT 
*
FROM trafc.electronic_count_data_type_10 t10
left join traf_lu.vehicle_classes_scheme_08 c on c.id = t10.vehicle_class_code_primary_scheme
where t10.header_id = 'bad666cb-0815-459a-b0b0-e17172b7ed17'
"""

In [2]:
AXLE_SPACING_SELECT_QRY = """SELECT 
t10.id,
        t10.header_id, 
        t10.start_datetime,
        t10.edit_code,
        t10.vehicle_class_code_primary_scheme, 
        t10.vehicle_class_code_secondary_scheme,
        t10.direction,
        t10.axle_count,
        axs.axle_spacing_number,
        axs.axle_spacing_cm
        FROM trafc.electronic_count_data_type_10 t10
        left join trafc.traffic_e_type10_axle_spacing axs ON axs.type10_id = t10.data_id
where t10.header_id = 'bad666cb-0815-459a-b0b0-e17172b7ed17'
"""

In [4]:
WHEEL_MASS_SELECT_QRY = """SELECT 
t10.id,
t10.header_id, 
t10.start_datetime,
t10.edit_code,
t10.vehicle_class_code_primary_scheme, 
t10.vehicle_class_code_secondary_scheme,
t10.direction,
wm.wheel_mass_number,
wm.wheel_mass,
vm.kg as vehicle_mass_limit_kg,
sum(wm.wheel_mass) over(partition by t10.id) as gross_mass
FROM trafc.electronic_count_data_type_10 t10
left join trafc.traffic_e_type10_wheel_mass wm ON wm.type10_id = t10.data_id
left join traf_lu.gross_vehicle_mass_limits vm on vm.number_of_axles = t10.axle_count
where t10.header_id = 'bad666cb-0815-459a-b0b0-e17172b7ed17'
"""

In [95]:
df = pd.read_sql_query(SELECT_QRY,config.ENGINE)

In [3]:
df2 = pd.read_sql_query(AXLE_SPACING_SELECT_QRY,config.ENGINE)

In [96]:
df3 = pd.read_sql_query(WHEEL_MASS_SELECT_QRY,config.ENGINE)

In [6]:
mean_equivalent_axle_mass_per_class_primary = (df.groupby(['id', 'vehicle_class_code_primary_scheme']).sum()*2).groupby(['vehicle_class_code_primary_scheme']).mean()['wheel_mass'].round(2)
mean_equivalent_axle_mass = (df.groupby(pd.Grouper(key='id')).mean()*2).mean().round(2)
mean_axle_spacing_number = (df.groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'].round()
number_of_vehicles_per_class_primary = df2.groupby(['vehicle_class_code_primary_scheme']).count()['id']


In [22]:
df3['e80_per_axle'] = (df3['wheel_mass']/8200)**4.2

In [33]:
(df3.groupby(['id']).sum().mean()['e80_per_axle']*100).round(2)

81.25

In [99]:
df3

,id,header_id,start_datetime,edit_code,vehicle_class_code_primary_scheme,vehicle_class_code_secondary_scheme,direction,wheel_mass_number,wheel_mass,vehicle_mass_limit_kg,gross_mass
0,50274702,bad666cb-0815-459a-b0b0-e17172b7ed17,2021-12-01 00:00:20.500,None,16,4,P,1.0,4341.0,56000.0,31791.0
1,50274702,bad666cb-0815-459a-b0b0-e17172b7ed17,2021-12-01 00:00:20.500,None,16,4,P,4.0,4298.0,56000.0,31791.0
2,50274702,bad666cb-0815-459a-b0b0-e17172b7ed17,2021-12-01 00:00:20.500,None,16,4,P,3.0,4544.0,56000.0,31791.0
3,50274702,bad666cb-0815-459a-b0b0-e17172b7ed17,2021-12-01 00:00:20.500,None,16,4,P,5.0,4529.0,56000.0,31791.0
4,50274702,bad666cb-0815-459a-b0b0-e17172b7ed17,2021-12-01 00:00:20.500,None,16,4,P,6.0,4457.0,56000.0,31791.0
...,...,...,...,...,...,...,...,...,...,...,...
279969,50412134,bad666cb-0815-459a-b0b0-e17172b7ed17,2021-12-31 23:59:21.400,None,16,4,P,3.0,5614.0,56000.0,37086.0
279970,50412134,bad666cb-0815-459a-b0b0-e17172b7ed17,2021-12-31 23:59:21.400,None,16,4,P,2.0,5513.0,56000.0,37086.0
279971,50412134,bad666cb-0815-459a-b0b0-e17172b7ed17,2021-12-31 23:59:21.400,None,16,4,P,7.0,4558.0,56000.0,37086.0
279972,50412134,bad666cb-0815-459a-b0b0-e17172b7ed17,2021-12-31 23:59:21.400,None,16,4,P,6.0,5354.0,56000.0,37086.0


In [93]:
((((df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='N')]['wheel_mass']/8200)**4.2).sum().round()/df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='N')].count()[0])*100).round(2)

17.33

In [77]:
((((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']/8200)**4.2).sum().round()/((df3['wheel_mass']/8200)**4.2).sum().round())*100).round(2)

5.86

In [19]:
df2.groupby(pd.Grouper(key='id')).count().count()[0]

59502